In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, roc_curve, auc
from lightgbm import LGBMClassifier
from tqdm import tqdm

# Read the data
train_origin = pd.read_csv('/Users/jaesolshin/내 드라이브/2024-2/Google ML Bootcamp2024/data/playground1/train.csv')

In [2]:
from category_encoders import TargetEncoder

# 데이터 샘플링
train = train_origin.sample(frac=0.01, random_state = 42)

# 예측에 필요 없는 'id'와 'Annual_Premium' 변수를 드롭
train = train.drop(columns=['id'])

# 범주형 변수 인코딩
def encoding(train):
    gender_mapping = {'Male': 0, 'Female': 1}
    vehicle_age_mapping = {'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2}
    vehicle_damage_mapping = {'No': 0, 'Yes': 1}

    train['Gender'] = train['Gender'].map(gender_mapping)
    train['Vehicle_Age'] = train['Vehicle_Age'].map(vehicle_age_mapping)
    train['Vehicle_Damage'] = train['Vehicle_Damage'].map(vehicle_damage_mapping)

    return train

train = encoding(train)

# 범주형 변수 타겟 인코딩
cat_columns = ['Region_Code', 'Policy_Sales_Channel', 'Vintage']
train.loc[:,cat_columns] = train.loc[:,cat_columns].astype('category')

target_encoder = TargetEncoder()
train[cat_columns] = target_encoder.fit_transform(train[cat_columns],train['Response'])

#수치형 변수: Age, Annual Premium -> normalize
scaler = MinMaxScaler()
num_columns = ['Age', 'Annual_Premium']
train[num_columns] = scaler.fit_transform(train[num_columns])

# 예측변수 분리 및 train, valid set 분리
X = train.drop(['Response'], axis=1)
y = train['Response']

In [4]:
#pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [8]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score

# 목적 함수 정의
def objective(trial):
    param = {
        'num_leaves': trial.suggest_categorical('num_leaves', [20, 31, 50, 100]),
        'max_depth': trial.suggest_int('max_depth', -1, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
        'n_estimators': trial.suggest_categorical('n_estimators', [100, 200, 300]),
        'lambda_l1': trial.suggest_uniform('lambda_l1', 0.0, 2.0),
        'lambda_l2': trial.suggest_uniform('lambda_l2', 0.0, 2.0)
    }
    clf = LGBMClassifier(**param)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(clf, X, y, cv=cv, scoring='roc_auc').mean()
    return score

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# 최적의 하이퍼파라미터 출력
print("Best parameters found by Optuna:", study.best_params)
print("Best ROC AUC score found by Optuna:", study.best_value)

[I 2024-07-22 22:49:57,575] A new study created in memory with name: no-name-11432f85-55cb-416e-bf44-c36b2c9ce13d


/var/folders/fx/7qldcfl92nb_g9hqp_lrssxm0000gn/T/ipykernel_90945/3462357108.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
/var/folders/fx/7qldcfl92nb_g9hqp_lrssxm0000gn/T/ipykernel_90945/3462357108.py:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'lambda_l1': trial.suggest_uniform('lambda_l1', 0.0, 2.0),
/var/folders/fx/7qldcfl92nb_g9hqp_lrssxm0000gn/T/ipykernel_90945/3462357108.py:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'lambda_l2': trial.suggest_uniform('

[LightGBM] [Warning] lambda_l1 is set=1.0801222425490595, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.0801222425490595
[LightGBM] [Warning] lambda_l2 is set=1.9105228704792698, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.9105228704792698
[LightGBM] [Warning] lambda_l1 is set=1.0801222425490595, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.0801222425490595
[LightGBM] [Warning] lambda_l2 is set=1.9105228704792698, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.9105228704792698
[LightGBM] [Info] Number of positive: 11313, number of negative: 80725
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003396 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 709
[LightGBM] [Info] Number of data points in the train set: 92038, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: 

[I 2024-07-22 22:50:25,654] Trial 0 finished with value: 0.8741716796087227 and parameters: {'num_leaves': 31, 'max_depth': 14, 'learning_rate': 0.04482775576860681, 'n_estimators': 300, 'lambda_l1': 1.0801222425490595, 'lambda_l2': 1.9105228704792698}. Best is trial 0 with value: 0.8741716796087227.
/var/folders/fx/7qldcfl92nb_g9hqp_lrssxm0000gn/T/ipykernel_90945/3462357108.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
/var/folders/fx/7qldcfl92nb_g9hqp_lrssxm0000gn/T/ipykernel_90945/3462357108.py:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'lambda_l1': trial.suggest_uniform('lambda_l1', 0.0, 2.0),
/var/f

[LightGBM] [Warning] lambda_l1 is set=1.5279081650838997, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.5279081650838997
[LightGBM] [Warning] lambda_l2 is set=1.9358821726545732, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.9358821726545732
[LightGBM] [Warning] lambda_l1 is set=1.5279081650838997, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.5279081650838997
[LightGBM] [Warning] lambda_l2 is set=1.9358821726545732, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.9358821726545732
[LightGBM] [Info] Number of positive: 11313, number of negative: 80725
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003726 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 709
[LightGBM] [Info] Number of data points in the train set: 92038, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: 

...

[LightGBM] [Warning] lambda_l1 is set=1.646422533578869, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.646422533578869
[LightGBM] [Warning] lambda_l2 is set=0.6108124098179399, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.6108124098179399


/var/folders/fx/7qldcfl92nb_g9hqp_lrssxm0000gn/T/ipykernel_90945/3462357108.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
/var/folders/fx/7qldcfl92nb_g9hqp_lrssxm0000gn/T/ipykernel_90945/3462357108.py:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'lambda_l1': trial.suggest_uniform('lambda_l1', 0.0, 2.0),
/var/folders/fx/7qldcfl92nb_g9hqp_lrssxm0000gn/T/ipykernel_90945/3462357108.py:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'lambda_l2': trial.suggest_uniform('

[LightGBM] [Warning] lambda_l1 is set=0.45529966709282393, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.45529966709282393
[LightGBM] [Warning] lambda_l2 is set=0.1583875757630056, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1583875757630056
[LightGBM] [Warning] lambda_l1 is set=0.45529966709282393, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.45529966709282393
[LightGBM] [Warning] lambda_l2 is set=0.1583875757630056, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1583875757630056
[LightGBM] [Info] Number of positive: 11313, number of negative: 80725
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.073465 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 709
[LightGBM] [Info] Number of data points in the train set: 92038, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.122917 -> initscore=-1.965096
[LightGBM] [Info] Start

[I 2024-07-22 23:09:10,223] Trial 49 finished with value: 0.8619739347558666 and parameters: {'num_leaves': 31, 'max_depth': 1, 'learning_rate': 0.016982784732478537, 'n_estimators': 300, 'lambda_l1': 0.45529966709282393, 'lambda_l2': 0.1583875757630056}. Best is trial 32 with value: 0.8752334244960988.


Best parameters found by Optuna: {'num_leaves': 31, 'max_depth': 4, 'learning_rate': 0.09052003360634207, 'n_estimators': 200, 'lambda_l1': 0.3182477714588782, 'lambda_l2': 0.4879151959075913}
Best ROC AUC score found by Optuna: 0.8752334244960988


Best parameters found by Optuna: {'num_leaves': 31, 'max_depth': 4, 'learning_rate': 0.09052003360634207, 'n_estimators': 200, 'lambda_l1': 0.3182477714588782, 'lambda_l2': 0.4879151959075913}
Best ROC AUC score found by Optuna: 0.8752334244960988